In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port=5433)
print("connection successful")
# cur = conn.cursor()
query="""SELECT * FROM eur_usd_final ORDER BY datetime DESC;"""
# cur.execute(query=query)
# data = cur.fetchall()
# cur.close()
df = pd.read_sql(query, conn)
conn.close()
print(df.head())

connection successful
     id    datetime     open     high      low    close  close_ratio_2  \
0  5020  2026-01-03  1.17234  1.17237  1.17159  1.17204       0.999868   
1  5021  2026-01-02  1.17442  1.17653  1.17139  1.17235       0.999118   
2  5022  2026-01-01  1.17464  1.17506  1.17429  1.17442       0.999472   
3  5023  2025-12-31  1.17464  1.17583  1.17210  1.17566       1.000443   
4  5024  2025-12-30  1.17675  1.17801  1.17459  1.17462       0.999098   

   close_ratio_5  close_ratio_60  close_ratio_250  close_ratio_1000  
0       0.998485        1.006888         1.021131          1.078417  
1       0.997950        1.007282         1.021864          1.078783  
2       0.998891        1.009216         1.024123          1.080763  
3       0.999412        1.010482         1.025655          1.081973  
4       0.998076        1.009814         1.025201          1.081084  


/var/folders/5l/17wd4myj387b056cpgkkpxth0000gn/T/ipykernel_47419/3458372560.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
df.head()

,id,datetime,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000
0,5020,2026-01-03,1.17234,1.17237,1.17159,1.17204,0.999868,0.998485,1.006888,1.021131,1.078417
1,5021,2026-01-02,1.17442,1.17653,1.17139,1.17235,0.999118,0.997950,1.007282,1.021864,1.078783
2,5022,2026-01-01,1.17464,1.17506,1.17429,1.17442,0.999472,0.998891,1.009216,1.024123,1.080763
3,5023,2025-12-31,1.17464,1.17583,1.17210,1.17566,1.000443,0.999412,1.010482,1.025655,1.081973
4,5024,2025-12-30,1.17675,1.17801,1.17459,1.17462,0.999098,0.998076,1.009814,1.025201,1.081084


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4014 entries, 0 to 4013
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                4014 non-null   int64  
 1   datetime          4014 non-null   object 
 2   open              4014 non-null   float64
 3   high              4014 non-null   float64
 4   low               4014 non-null   float64
 5   close             4014 non-null   float64
 6   close_ratio_2     4014 non-null   float64
 7   close_ratio_5     4014 non-null   float64
 8   close_ratio_60    4014 non-null   float64
 9   close_ratio_250   4014 non-null   float64
 10  close_ratio_1000  4014 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 345.1+ KB


In [5]:
df = df.drop(columns=["id"])
df["tomorrow"] = df["close"].shift(1)
df = df.dropna()
df["target"] = (df["close"] < df["tomorrow"]).astype(int)
df = df.set_index("datetime")
df.head()

,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
datetime,,,,,,,,,,,
2026-01-02,1.17442,1.17653,1.17139,1.17235,0.999118,0.997950,1.007282,1.021864,1.078783,1.17204,0
2026-01-01,1.17464,1.17506,1.17429,1.17442,0.999472,0.998891,1.009216,1.024123,1.080763,1.17235,0
2025-12-31,1.17464,1.17583,1.17210,1.17566,1.000443,0.999412,1.010482,1.025655,1.081973,1.17442,0
2025-12-30,1.17675,1.17801,1.17459,1.17462,0.999098,0.998076,1.009814,1.025201,1.081084,1.17566,1
2025-12-29,1.17695,1.17889,1.17506,1.17674,0.999813,0.999220,1.011789,1.027512,1.083093,1.17462,0


In [6]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=42)

test = df.iloc[:100]
train = df.iloc[100:]

predictors = ["close", "open", "high", "low"]
model.fit(train[predictors], train["target"])

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",100
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metr

In [9]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)

In [11]:
precision_score(test["target"], preds)

0.421875

In [12]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=42)

test_data = df.iloc[:500]
train_cv_data = df.iloc[500:]

predictors = df.columns.drop(["target", "tomorrow"])

In [14]:
predictors

Index(['open', 'high', 'low', 'close', 'close_ratio_2', 'close_ratio_5',
       'close_ratio_60', 'close_ratio_250', 'close_ratio_1000'],
      dtype='object')

In [15]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import precision_score

tscv = TimeSeriesSplit(
    n_splits=5,
    test_size=400,
    gap=1
)

X = train_cv_data[predictors]
y = train_cv_data["target"]

precision_scores = []

for i, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(
        X=X_tr,
        y=y_tr
    )

    preds = model.predict(X_val)
    fold_score = precision_score(y_val, preds)
    precision_scores.append(fold_score)

    print(f"Fold {i+1} Precision Score: {fold_score:.4f}") 

print(f"\n✅ Avg CV Precision Score: {np.mean(precision_scores):.4f} ± {np.std(precision_scores):.4f}")


Fold 1 Precision Score: 0.5426
Fold 2 Precision Score: 0.5652
Fold 3 Precision Score: 0.5030
Fold 4 Precision Score: 0.5466
Fold 5 Precision Score: 0.5818

✅ Avg CV Precision Score: 0.5479 ± 0.0264


In [16]:
test_predictions = model.predict(test_data[predictors])

test_score = precision_score(test_predictions, test_data["target"])

In [17]:
test_score

0.4406779661016949

In [18]:
import joblib
joblib.dump(model, 'forex_model.pkl')

['forex_model.pkl']

In [5]:
import psycopg2
import pandas as pd 

conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port=5433)
print("connection successful")
query="""SELECT * FROM eur_usd_train ORDER BY datetime DESC;"""
df = pd.read_sql(query, conn)
conn.close()
print(df.head())

connection successful
    id    datetime     open     high      low    close  close_ratio_2  \
0  480  2024-03-18  1.08826  1.09075  1.08808  1.08826       0.999963   
1  481  2024-03-15  1.08834  1.09016  1.08738  1.08834       0.996708   
2  482  2024-03-14  1.09553  1.09553  1.08841  1.09553       1.001257   
3  483  2024-03-13  1.09278  1.09493  1.09205  1.09278       0.999835   
4  484  2024-03-12  1.09314  1.09421  1.09032  1.09314       0.999470   

   close_ratio_5  close_ratio_60  close_ratio_250  close_ratio_1000  tomorrow  \
0       0.996931        1.000312         1.003004          0.975716   1.08739   
1       0.995902        1.000165         1.003107          0.975794   1.08826   
2       1.001259        1.006583         1.009726          0.982247   1.08834   
3       0.999766        1.004071         1.007233          0.979793   1.09553   
4       1.001411        1.004320         1.007597          0.980123   1.09278   

   target  
0       0  
1       0  
2       0  
3   

/var/folders/5l/17wd4myj387b056cpgkkpxth0000gn/T/ipykernel_92892/3329978031.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
df.head()

,id,datetime,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
0,480,2024-03-18,1.08826,1.09075,1.08808,1.08826,0.999963,0.996931,1.000312,1.003004,0.975716,1.08739,0
1,481,2024-03-15,1.08834,1.09016,1.08738,1.08834,0.996708,0.995902,1.000165,1.003107,0.975794,1.08826,0
2,482,2024-03-14,1.09553,1.09553,1.08841,1.09553,1.001257,1.001259,1.006583,1.009726,0.982247,1.08834,0
3,483,2024-03-13,1.09278,1.09493,1.09205,1.09278,0.999835,0.999766,1.004071,1.007233,0.979793,1.09553,1
4,484,2024-03-12,1.09314,1.09421,1.09032,1.09314,0.999470,1.001411,1.004320,1.007597,0.980123,1.09278,0


In [7]:
df = df.set_index("datetime")
df 

,id,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
datetime,,,,,,,,,,,,
2024-03-18,480,1.08826,1.09075,1.08808,1.08826,0.999963,0.996931,1.000312,1.003004,0.975716,1.08739,0
2024-03-15,481,1.08834,1.09016,1.08738,1.08834,0.996708,0.995902,1.000165,1.003107,0.975794,1.08826,0
2024-03-14,482,1.09553,1.09553,1.08841,1.09553,1.001257,1.001259,1.006583,1.009726,0.982247,1.08834,0
2024-03-13,483,1.09278,1.09493,1.09205,1.09278,0.999835,0.999766,1.004071,1.007233,0.979793,1.09553,1
2024-03-12,484,1.09314,1.09421,1.09032,1.09314,0.999470,1.001411,1.004320,1.007597,0.980123,1.09278,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-16,3997,1.30130,1.31130,1.29789,1.30151,1.000900,1.012235,1.015854,0.959013,0.938611,1.30760,1
2010-09-15,3998,1.29911,1.30359,1.29592,1.29917,1.004628,1.014885,1.014923,0.956866,0.936955,1.30151,1
2010-09-14,3999,1.28717,1.30180,1.28329,1.28720,1.006057,1.010486,1.006453,0.947585,0.928355,1.29917,1


In [8]:
df = df.sort_index(ascending=True)
df

,id,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
datetime,,,,,,,,,,,,
2010-09-10,4001,1.26899,1.27444,1.26470,1.26931,0.998470,0.993735,0.993757,0.933419,0.915486,1.27170,1
2010-09-13,4000,1.27189,1.28879,1.27170,1.27170,1.000941,0.998375,0.995108,0.935701,0.917197,1.28720,1
2010-09-14,3999,1.28717,1.30180,1.28329,1.28720,1.006057,1.010486,1.006453,0.947585,0.928355,1.29917,1
2010-09-15,3998,1.29911,1.30359,1.29592,1.29917,1.004628,1.014885,1.014923,0.956866,0.936955,1.30151,1
2010-09-16,3997,1.30130,1.31130,1.29789,1.30151,1.000900,1.012235,1.015854,0.959013,0.938611,1.30760,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12,484,1.09314,1.09421,1.09032,1.09314,0.999470,1.001411,1.004320,1.007597,0.980123,1.09278,0
2024-03-13,483,1.09278,1.09493,1.09205,1.09278,0.999835,0.999766,1.004071,1.007233,0.979793,1.09553,1
2024-03-14,482,1.09553,1.09553,1.08841,1.09553,1.001257,1.001259,1.006583,1.009726,0.982247,1.08834,0


In [9]:
df = df.drop(columns=["id"])
df

,open,high,low,close,close_ratio_2,close_ratio_5,close_ratio_60,close_ratio_250,close_ratio_1000,tomorrow,target
datetime,,,,,,,,,,,
2010-09-10,1.26899,1.27444,1.26470,1.26931,0.998470,0.993735,0.993757,0.933419,0.915486,1.27170,1
2010-09-13,1.27189,1.28879,1.27170,1.27170,1.000941,0.998375,0.995108,0.935701,0.917197,1.28720,1
2010-09-14,1.28717,1.30180,1.28329,1.28720,1.006057,1.010486,1.006453,0.947585,0.928355,1.29917,1
2010-09-15,1.29911,1.30359,1.29592,1.29917,1.004628,1.014885,1.014923,0.956866,0.936955,1.30151,1
2010-09-16,1.30130,1.31130,1.29789,1.30151,1.000900,1.012235,1.015854,0.959013,0.938611,1.30760,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12,1.09314,1.09421,1.09032,1.09314,0.999470,1.001411,1.004320,1.007597,0.980123,1.09278,0
2024-03-13,1.09278,1.09493,1.09205,1.09278,0.999835,0.999766,1.004071,1.007233,0.979793,1.09553,1
2024-03-14,1.09553,1.09553,1.08841,1.09553,1.001257,1.001259,1.006583,1.009726,0.982247,1.08834,0


In [10]:
df.to_csv("train_sample.csv")